# ipytone:
_____


Once that we have installed our ipytone library, we can import it:

Links:
[Library](https://github.com/benbovy/ipytone)
[Documentation](https://ipytone.readthedocs.io/en/latest/)

In [1]:
from my_framework import *

In [2]:
import ipytone

# Comments in python are written with this symbol #
# ipywidgets is going to permit us control the objects unsing some interactive widgets...
import ipywidgets as widgets

Let's create a basic synth:

In [ ]:
# we can specify some parameters in the constructor....
synth = ipytone.MonoSynth(volume=-7)

# we need to connect the synth to the destination also here in ipytone....
synth.to_destination()

ipytone exports a lot of tone.js functionalities to python, permitting us to use the same objects. 

Remember we saw the sequence object? Here we define a sequence and a callback function:

In [ ]:
# Notice we don't need brackets on functions anymore,
# python works with indentation...
def callback_whatever(time, note):
    synth.trigger_attack_release(note, 0.2, time=time)


sequence = ipytone.Sequence(
    callback=callback_whatever,
    events=["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"],
    subdivision="16n",
)

What happens if we want to start tone? (or, in this case ipytone)?

In [ ]:
ipytone.start()

As you see, we don't use it anymore here in ipytone. 

The transport instead, we continue using it.

Another thing: running a notebook cell is like executing a user action, so we can just invoke the transport start function and the sequence start function on a new cell:

In [ ]:
ipytone.transport.start()
sequence.start()

We can pause the transport and our sequences will automatically be interrupted:

In [ ]:
ipytone.transport.pause()

We always need to dispose our elements if we want to control well our objects:

In [ ]:
synth.dispose()
sequence.dispose()

We can also modify on the fly the values of the sequence:

In [ ]:
sequence.events=["A0", "A1", "A0", None, None, "G2", None, "A2"]

In [ ]:
sequence.events=["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"]

Connecting and disconecting on the fly:

In [ ]:
synth.disconnect(ipytone.destination)

connecting it again:

In [ ]:
synth.to_destination()

> Note: In tone.js and in Ipytone, we have the Param class and the Signal class. To modify their values, we need the value accessor. 

## Some exploration:
___

Let's first create some synth's and connect them to the speakers through some specific processing nodes:

[Channels](https://ipytone.readthedocs.io/en/latest/_api_generated/ipytone.Channel.html#ipytone.Channel) are handy to control the volume, the pan, and even mute a whole strip

In [ ]:
# I create a synth but i don't connect it to the destination. 
synth = ipytone.Synth(volume=-10)

# I create a channel and connect it to the destination:
chan = ipytone.Channel(pan=-0.5).to_destination()

# I connect the synth to the channel:
synth.connect(chan)

# I can hear the synth through the channel:
synth.trigger_attack_release('A4', 4)

In [ ]:
synth.volume.value = 0

In [ ]:
# I can also modify the channel's params on the fly:

import time

synth.trigger_attack_release('A4', 4)

time.sleep(1.4)

print("Now changing the pan of the channel")
chan.pan.value = 0

In [ ]:
synth.dispose()
chan.dispose()

### We now create two synths:
____
we can also test how the [PingPongDelay](https://ipytone.readthedocs.io/en/latest/search.html?q=PingPongDelay) effect affect the synth....

In [26]:
channel1 = ipytone.Channel(pan=-0.5)

channel1.to_destination()

filtr = ipytone.Filter(frequency=100)

synth = ipytone.MonoSynth(volume=-7)

synth.chain(filtr, channel1)



channel2 = ipytone.Channel(pan=0.2, channel_count=2).to_destination()

delay = ipytone.PingPongDelay(delay_time="16n", feedback=0.2)

perc_synth = ipytone.MembraneSynth(volume=-10).chain(delay, channel2)

We are now going to create other effects, like the [LFO](https://ipytone.readthedocs.io/en/latest/_api_generated/ipytone.LFO.html?highlight=LFO)

In [27]:
lfo = ipytone.LFO(frequency="4m", min=100, max=10_000)


lfo.connect(filtr.frequency).start()



# lfo2 = ipytone.LFO(frequency="8n", min=-200, max=200, type="triangle")
# lfo2.connect(perc_synth.detune).start()


perc_synth.pitch_decay = 0.02
delay.wet.value = 0.1
# lfo2.amplitude.value = 0.6

In [28]:
def clb(time, note):
    synth.trigger_attack_release(note, 0.2, time=time)
    perc_synth.trigger_attack_release(note, 0.05, time=time)

    
sequence = ipytone.Sequence(
    callback=clb,
    events=["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"],
    subdivision="16n",
)

In [29]:
ipytone.transport.start()
sequence.start()

Sequence(loop=True, loop_start=0.0, loop_end=0.0)

In [ ]:
lfo.stop()

In [ ]:
lfo.start()

Let's modify some values on the fly:

In [ ]:
# let's concentrate in the perc_synth then!

def clb2(time, note):
    perc_synth.trigger_attack_release(note, 0.05, time=time)
    
    
sequence.callback = clb2

In [ ]:
# Restore
sequence.callback = clb

In [ ]:
lfo.stop()

In [ ]:
lfo.start()

In [23]:
ipytone.transport.pause()

Transport()

#### Doing some more stuff:

In [8]:
import asyncio

In [36]:
ipytone.transport.start()


Transport()

In [9]:
# secs per minute / bits per minute = secs per bit.
secs_per_bit = 60 / ipytone.transport.bpm.value

bits_per_sec = ipytone.transport.bpm.value / 60

'''
In our sequence we have a loop interval of 16n, and in our sequence events we have 8 events.
So for each measure we loop twice around the sequence events. 

We now that a measure takes 4 secs when bpm is 120. 
This means that a measure corresponds to 8 bits in this configuration, (and maybe always)
So we multiply secs_per_bit * bits_per_measure to get secs_per_measure:
'''

bits_per_measure = 8

secs_per_measure =  bits_per_measure / bits_per_sec

m2 = secs_per_measure

In [10]:
m2

4.0

The following asynchornous function starts a new thread and inside that thread modifies the events array of the sequence and some other params with some delays that are synchronized with the secs_per_measure time:

In [11]:
import time

In [30]:
async def melody():
    """8x2 measures melody"""
    
    
    # 1 ---
    print('step 1', time.time())
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "A4"]
    await asyncio.sleep(m2)
    
    
    # 2 ---
    print('step 2', time.time())
    # the following line of code could be replaced by sequence.events[-1] = 'B4' and it would be equivalent. 
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "B4"]
    await asyncio.sleep(m2)
    
    
    
    # 3 ---
    print('step 3', time.time())
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "C4"]
    await asyncio.sleep(m2)
    
    
    # 4 ---
    print('step 4', time.time())
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "E4", "A4"]
    await asyncio.sleep(m2)
    
    
    
    # 5 ---
    print('step 5', time.time())
    sequence.events = ["A1", None, None, "A0", "E5", None, None, None]
    await asyncio.sleep(m2)
    
    
    
    # 6 ---
    print('step 6', time.time())
    synth.portamento = 0.08
    filtr.q.ramp_to(6, "4m")
    #lfo2.amplitude.ramp_to(1, "4m")
    await asyncio.sleep(m2)
    
    
    
    # 7 ---
    print('step 7', time.time())
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "G5", "A4"]
    await asyncio.sleep(m2)
    
    
    # 8 ---
    print('step 8', time.time())
    delay.wet.ramp_to(0.8, "2m")
    synth.portamento = 0.1
    await asyncio.sleep(m2)
    
    
    # 1 --- Go back to original values (quicker)
    print('Restoring params', ipytone.transport.position)
    synth.portamento = 0
    filtr.q.ramp_to(1, "4n")
    delay.wet.ramp_to(0.1, "4n")
    # This is an esception because the original value of the amplitude was 1.0
    lfo.amplitude.ramp_to(0.1, "4n")
    sequence.events = ["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"]


In [31]:
loop = asyncio.get_event_loop()

In [37]:
loop.create_task(melody());

step 1 1671811290.015097
step 2 1671811294.016622
step 3 1671811298.0257375


In [38]:
print('hello')

hello
step 4 1671811302.0365002
step 5 1671811306.0391548
step 6 1671811310.046606
step 7 1671811314.049729
step 8 1671811318.051944
Restoring params 75:2:1.92


In [35]:
def sync_melody():
    """8x2 measures melody"""
    # 1 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "A4"]
    time.sleep(m2)
    
    
    # 2 ---
    # the following line of code could be replaced by sequence.events[-1] = 'B4' and it would be equivalent. 
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "B4"]
    time.sleep(m2)
    
    
    
    # 3 ---
    sequence.events = ["A1", "A2", "A0", None, "E4", "A3", "E4", "C4"]
    time.sleep(m2)
    
    
    # 4 ---
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "E4", "A4"]
    time.sleep(m2)
    
    
    
    # 5 ---
    sequence.events = ["A1", None, None, "A0", "E5", None, None, None]
    time.sleep(m2)
    
    
    
    # 6 ---
    synth.portamento = 0.08
    filtr.q.ramp_to(6, "4m")
    #lfo2.amplitude.ramp_to(1, "4m")
    time.sleep(m2)
    
    
    
    # 7 ---
    sequence.events = ["A1", "A2", None, "A0", "E5", None, "G5", "A4"]
    time.sleep(m2)
    
    
    # 8 ---
    delay.wet.ramp_to(0.8, "2m")
    synth.portamento = 0.1
    time.sleep(m2)
    
    
    # 1 --- Go back to original values (quicker)
    synth.portamento = 0
    filtr.q.ramp_to(1, "4n")
    delay.wet.ramp_to(0.1, "4n")
    # This is an esception because the original value of the amplitude was 1.0
    lfo.amplitude.ramp_to(0.1, "4n")
    sequence.events = ["A0", "A1", "A0", None, "F#2", "G2", "G#2", "A2"]


In [39]:
sync_melody()

In [25]:
synth.dispose()
filtr.dispose()
lfo.dispose()
#lfo2.dispose()
channel1.dispose()
perc_synth.dispose()
delay.dispose()
channel2.dispose()

sequence.dispose()

Sequence(disposed=True, loop=True, loop_start=0.0, loop_end=0.0)

# Widgets: 

In [33]:
ipytone.transport.pause()

Transport()

In [ ]:
ipytone.transport.start()

In [40]:
pan = widgets.FloatSlider(
        value=channel1.pan.value, min=-1, max=1,
        layout=widgets.Layout(width="200px")
    )

widgets.jslink((pan, "value"), (channel1.pan, "value"))

Link(source=(FloatSlider(value=-0.5, layout=Layout(width='200px'), max=1.0, min=-1.0), 'value'), target=(Param(value=-0.5, units='audioRange'), 'value'))

In [41]:
pan

FloatSlider(value=-0.5, layout=Layout(width='200px'), max=1.0, min=-1.0)

In [43]:
def create_channel(node):
    # panner
    pan = widgets.FloatSlider(
        value=node.pan.value, min=-1, max=1,
        layout=widgets.Layout(width="200px")
    )
    widgets.jslink((pan, "value"), (node.pan, "value"))
    
    # solo / mute buttons
    solo = widgets.ToggleButton(value=False, description="Solo")
    mute = widgets.ToggleButton(value=False, description="Mute")

    def node_solo(change):
        node.solo = change['new']
        
    def node_mute(change):
        node.mute = change["new"]
        
    solo.observe(node_solo, names='value')
    mute.observe(node_mute, names='value')
    
    # fader
    fader = widgets.FloatSlider(
        value=0, min=-30, max=4, orientation="vertical"
    )
    widgets.jslink((fader, "value"), (node.volume, "value"))

    # L/R VU meters
    vu_left = widgets.FloatProgress(
        min=0, max=0.4, orientation="vertical"
    )
    vu_right = widgets.FloatProgress(
        min=0, max=0.4, orientation="vertical"
    )
    split = ipytone.Split()
    node.connect(split)
    
    meter_left = ipytone.Meter(normal_range=True)
    split.connect(meter_left, 0, 0)
    meter_left.schedule_jsdlink((vu_left, "value"), transport=True)
    meter_right = ipytone.Meter(normal_range=True)
    split.connect(meter_right, 1, 0)
    meter_right.schedule_jsdlink((vu_right, "value"), transport=True)
    
    # layout
    fader_vus = widgets.HBox([fader, vu_left, vu_right])
    return widgets.VBox([pan, solo, mute, fader_vus])

widgets.HBox([create_channel(channel1), create_channel(channel2)])